
# Теория Кодирования
## Лабораторная работа 5

От студента группы **6405**-010302
**Слеповичева Святослава**


---

### Задание
#### 5.1 Написать функцию формирования порождающей матрицы кода Рида-Маллера (r,m) в каноническом виде для произвольно заданных r и m


In [60]:
import numpy as np
import math
from itertools import product, combinations

In [61]:
# Выберем "произвольно" заданные числа, какие удобнее
# кодовое расстояние = 4, а значит по формуле количество возможно исправленных ошибок:
# (4 - 1) / 2 = 1 (округляем в меньшую сторону)

r, m = 2, 4

standart_order = np.array(list(product(*[[0, 1] for _ in range(m)])))
standart_order = np.array([el[::-1]for el in standart_order])

master_set = [i for i in range(m)]
I = [[*combinations(master_set, i)] for i in range(1, m + 1) if i <= r]
temp_arr = []

for el in I:
    temp_arr.extend(el)

I = temp_arr.copy()
I.insert(0, ())

print(standart_order)
print(I)

[[0 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 1 0 0]
 [0 0 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [1 1 1 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 0 1]
 [1 1 0 1]
 [0 0 1 1]
 [1 0 1 1]
 [0 1 1 1]
 [1 1 1 1]]
[(), (0,), (1,), (2,), (3,), (0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [62]:
v = []
for el in I:
    v_i = []
    for u in standart_order:
        if el == ():
            v_i.append(1)
        else:
            if sum([item for idx, item in enumerate(u) if idx in el]) == 0:
                v_i.append(1)
            else:
                v_i.append(0)
    v.append(v_i)

G = np.zeros((len(v), 2**m), dtype=int)
G[0, :] = v[0]

idx = 1
for j in range(1, r + 1):
    temp_arr = v[idx: idx + math.comb(m, j)]

    for i in range(len(temp_arr)):
        k = i + 1
        while (k < len(temp_arr)):
            for l in range(2**m - 1, 0, -1):
                if (temp_arr[i][l] != temp_arr[k][l]):
                    if(temp_arr[i][l] == 1):
                        temp_arr[i], temp_arr[k] = temp_arr[k], temp_arr[i]
                        k = i
                    break
            k += 1
    G[idx: idx + math.comb(m, j)] = temp_arr.copy()
    idx += math.comb(m, j)
G

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]])

---
#### 5.2. Реализовать алгоритм мажоритарного декодирования для кода Рида-Маллера

In [63]:
def major_decode_algr(w, r, m):

    cannot_be_decoded = False
    i = r
    wj = w.copy()

    ms = dict()
    while (i >= 0):
        J = [*combinations(master_set, i)]
        for el in J:
            v = []
            _J = set(el)
            _master_set = set(master_set)
            Jc = _master_set - _J

            Hj = []
            for item in standart_order:
                ssum = 0
                for j in range(len(item)):
                    if j in el:
                        ssum += item[j]
                if ssum == 0:
                    Hj.append(item)

            for t in Hj:
                v_Jc_t = []
                for word in standart_order:
                    s = ((np.array(word) + np.array(t)) % 2)
                    if sum([item for idx, item in enumerate(s) if idx in Jc]) == 0:
                        v_Jc_t.append(1)
                    else:
                        v_Jc_t.append(0)
                v.append(np.array(v_Jc_t, dtype=int))
            zeros_count = 0
            ones_count = 0

            border = 2 ** (m - i - 1)
            for x in v:
                if (wj @ x) % 2 == 0: zeros_count += 1
                else: ones_count += 1

                if (zeros_count > border) and (not((zeros_count > border) and (ones_count > border))):
                    ms[el] = 0
                elif (ones_count > border) and (not((zeros_count > border) and (ones_count > border))):
                    ms[el] = 1
                elif (zeros_count > border - 1) and (ones_count > border - 1):
                    cannot_be_decoded = True
                    return ms, cannot_be_decoded

        for el in J:
            vj = []
            for word in standart_order:
                if sum([item for idx, item in enumerate(word) if idx in el]) == 0:
                    vj.append(1)
                else:
                    vj.append(0)
            wj += (ms[el] * np.array(vj))
            wj %= 2

            if sum(wj) <= 2**(m - r - 1) - 1:
                I = [[*combinations(master_set, i)] for i in range(1, m + 1) if i <= r]
                temp_arr = []

                for el in I:
                    temp_arr.extend(el)

                I = temp_arr.copy()
                I.insert(0, ())

                absent = set(I) - set(ms.keys())
                for l in absent:
                    ms[l] = 0

                word = []
                for el in ms:
                    word.append(ms[el])
                return np.array(word), cannot_be_decoded
        i -= 1

        word = []
        for el in ms:
            word.append(ms[el])

    return np.array(word), cannot_be_decoded

---
#### 5.3. Провести экспериментальную проверку алгоритма декодирования для кода Рида-Маллера (2,4).

In [64]:
# Однократная ошибка
r, m = 2, 4

word = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])
send = (word @ G) % 2
send[0] ^= 1

source_word, decode_flag = major_decode_algr(send, r, m)
if(decode_flag):
    print("Error can not be fixed!")
else:
    print(f"Source word: {source_word}")
    print(all(source_word == word))

Source word: [0 1 0 0 0 0 0 0 0 1 0]
True


In [65]:
# Двукратная ошибка
send = (word @ G) % 2
send[0] ^= 1
send[1] ^= 1

source_word, decode_flag = major_decode_algr(send, r, m)

if(decode_flag):
    print("Error can not be fixed!")
else:
    print(f"Source word: {source_word}")
    print(all(source_word == word))

Error can not be fixed!
